In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import time

In [2]:
# Load dataset
df = pd.read_csv("../sim_og.csv")

In [3]:
X = df[['a', 'q', 'delta_e']].values
y = df[['Cd']].values

In [4]:
# Data split (random_state changed every run for variation)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42)

In [5]:
# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32)

In [6]:
class ELM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ELM, self).__init__()
        self.hidden_size = hidden_size
        self.input_weights = torch.nn.Parameter(torch.randn(input_size, hidden_size) * 0.1, requires_grad=False)
        self.biases = torch.nn.Parameter(torch.randn(hidden_size) * 0.1, requires_grad=False)
        self.output_weights = None  # To be determined analytically

    def forward(self, x):
        H = torch.sigmoid(torch.add(torch.matmul(x, self.input_weights), self.biases))
        return H

In [7]:
def train_elm(model, X, T):
    start_time = time.time()  
    with torch.no_grad():
        H = model(X)
        H_pseudo_inverse = torch.pinverse(H)
        model.output_weights = torch.matmul(H_pseudo_inverse, T)
    end_time = time.time()  
    print(f"Training Time: {end_time - start_time:.4f} seconds")

def predict(model, X):
    start_time = time.time() 
    with torch.no_grad():
        H = model(X)
        predictions = torch.matmul(H, model.output_weights)
        end_time = time.time() 
        print(f"Prediction Time: {end_time - start_time:.4f} seconds")
        return predictions

In [8]:
# Model Configuration
input_size = 3  # 28x28 pixels
hidden_size = 20  # Number of hidden neurons
output_size = 1  # Number of output classes

In [9]:
model = ELM(input_size, hidden_size, output_size)

In [10]:
# Training
train_elm(model, X_train, y_train)

Training Time: 0.1179 seconds


In [11]:
# Testing
y_pred = predict(model, X_test)

# Evaluation
mse = mean_squared_error(y_test.numpy(), y_pred.numpy())
r2 = r2_score(y_test.numpy(), y_pred.numpy())

print(f"MSE: {mse:}")
print(f"R² Score: {r2:}")

Prediction Time: 0.0047 seconds
MSE: 1.5273641906787816e-08
R² Score: 0.9999998211860657


In [12]:
# Save the trained model
torch.save({
    'input_weights': model.input_weights.data,
    'biases': model.biases.data,
    'output_weights': model.output_weights
}, 'trained_elm.pth')
print("Model saved as trained_elm.pth")

Model saved as trained_elm.pth
